In [ ]:
using Pkg; Pkg.activate(".")
using SparseArrays, LinearAlgebra, Ferrite, 
      WGLMakie, StaticArrays, ForwardDiff

include("tools.jl");

# P1 Finite Element Method 

We solve 

$$
\begin{aligned}
  - \Delta u &= f, \quad \Omega, \\ 
    u &= 0, \quad \partial \Omega
    \end{aligned}
$$

using a P1 finite element method. The weak form is : Find $u \in H^1_0(\Omega)$ such that 

$$
    \int_\Omega \nabla u \cdot \nabla v \,dx = \int_\Omega f v \,dx \qquad \forall v \in H^1_0.
$$

For the definition of the conforming P1 finite element space see the class notes. This gives us the space $V_h = P_1(\mathcal{T}_h) \cap H^1_0$ where $\mathcal{T}_h$ is a regular triangulation of $\Omega$. 

For example, the following short snippet generated a triangulation of the square $\Omega = (-1, 1)^2$. But in general, we should import meshes from a sophisticated mesh generator such as GMSH.

In [ ]:
grid = generate_grid(Triangle, (10, 10))

In [ ]:
trimesh(grid)

We can quickly learn how the triangulation is stored by inspecting the fields of `grid`. We see that `grid.nodes` is a list of positions and `grid.cells` is a list of triangles with each triangle specified by the indices of the nodes. This is all the information we need to proceed for now. There is a lot more information in `grid` that we don't require at the moment. 

In [ ]:
@show grid.nodes[1]
@show grid.cells[1];

In [ ]:
# get some helpful auxiliary information, namely the 
# list of boundary nodes. 
nnodes = length(grid.nodes)
boundary = findall(node -> norm(node.x, Inf) == 1, grid.nodes)
interior = setdiff(1:nnodes, boundary);

After expressing FE functions in terms of a nodal basis, $u_h = \sum_i U_i \psi_i$, we need to compute $A_{ij} = a(\psi_i, \psi_j)$ and $b_i = \ell(\psi_i)$. This represents the bilinear and linear forms as 

$$
  a(u_h, v_h) = V^T A U, \qquad \ell(v_h) = V^T b.
$$

This is implement in the main assembly loop as follows. See the class notes for the details of the implemented steps.

In [ ]:
function assemble_dirichlet_p1(grid, f)
    nnodes = length(grid.nodes)
    # pre-allocate : again this should be a sparse format 
    # but we continue to keep it simple for now.
    A = zeros(nnodes, nnodes)
    b = zeros(nnodes)
    
    for T in grid.cells 
        # local nodes
        XT = (grid.nodes[T.nodes[1]].x, 
              grid.nodes[T.nodes[2]].x, 
              grid.nodes[T.nodes[3]].x)
        # deformation matrix, volume element = abs(det(F)), grad transform
        F = [ XT[2] - XT[1] XT[3] - XT[1] ]
        volT = 0.5 * abs(det(F))
        F⁻ᵀ = pinv(F)'    # use SVD to compute F⁻¹ to be safe
        
        # element mid-point for quadrature
        #   ∫_T f(x) ψᵢ(x) dx = volT * f(ξ) * ψᵢ(ξ) + O(h⁴)
        ξ = sum(XT) / 3 
        fξ = f(ξ)
        
        # assemble the right-hand side
        for i = 1:3 
            nᵢ = T.nodes[i] 
            ψᵢ = 1/3   # ψᵢ(ξ) = 1/3 since ξ = midpoint
            b[nᵢ] += volT * fξ * ψᵢ
        end
        
        # assemble the stiffness matrix
        ∇ψ̂ = ( [-1.0, -1.0], [1.0, 0.0], [0.0, 1.0] )
        ∇ψ = ( F⁻ᵀ * ∇ψ̂[1], F⁻ᵀ * ∇ψ̂[2], F⁻ᵀ * ∇ψ̂[3] )
        for i = 1:3, j = 1:3 
            nᵢ = T.nodes[i]; nⱼ = T.nodes[j] 
            A[nᵢ, nⱼ] += volT * dot(∇ψ[i], ∇ψ[j])
        end         
    end
    
    return sparse(A), b 
end

In [ ]:
# now we use the same trick as in 1D. We generate 
# a DOF vector for ALL nodes but then solve the 
# equation only for the free nodes.
A, b = assemble_dirichlet_p1(grid, x -> 4.0)
U = zeros(nnodes)
U[interior] = A[interior, interior] \ b[interior];

In [ ]:
trisurf(grid, U; size = (400, 400))

Let's try a variation on this: Take $\Gamma_N$ to be the right face of $\Omega$ and solve the Dirichlet-Neumann problem 

$$
\begin{aligned}
  - \Delta u &= f, \quad \Omega \\ 
    u &= 0, \quad \Gamma_{\rm D} \\ 
    \nabla u \cdot \nu &= 0, \quad \Gamma_{\rm N} 
\end{aligned}
$$

The weak form becomes 

$$
  \int_\Omega \nabla u \cdot \nabla v dx = \int_\Omega f v dx \qquad \forall v \in H^1_{\Gamma_D},
$$

i.e. same as before but on a different space. So all we need to change is which nodes are clamped. 


In [ ]:
dirichlet_nodes = findall(node -> node.x[1] == -1 || abs(node.x[2]) == 1, grid.nodes)
free_nodes = setdiff(1:nnodes, dirichlet_nodes);
A, b = assemble_dirichlet_p1(grid, x -> 1.0)
U = zeros(nnodes)
U[free_nodes] = A[free_nodes, free_nodes] \ b[free_nodes];

In [ ]:
trisurf(grid, 4*U; size = (400, 400))

This gives a first simple FEM code to play with. We can now explore many things, more general $a_{ij}, a_0, f$, more general boundary conditions and so forth. For now we just add a simple convergence test?

For the Dirichlet problem the exact solution is not easy to find. So we change the problem a bit by specifying the solution, then analytically computing the force $f$. (we will actually use AD to make our lives a bit easier)

We can now compute the error w.r.t. to several norms. For the sake of simplicity, we do it in the $L^2$-norm and $H^1$-semi-norm. (these are also the ones we will analyze later)

In [ ]:
u_ex(x) = cos(pi*x[1]/2) * cos(pi*x[2]/2)
∇u_ex(x) = ForwardDiff.gradient(u_ex, x)
f_ex(x) = - tr(ForwardDiff.jacobian(∇u_ex, x));

# to be safe we can check that we did the right thing:
# x = randn(2)
# pi^2/2 * u_ex(x) ≈ f_ex(x)

In [ ]:

# the function `solve` puts together our script from above 
# and solves the same problem for different mesh sizes. 
# Here, N = number of elements along each face of the domain 
# to the corresponding mesh size is h = 1/N.
function solve(N)
    grid = generate_grid(Triangle, (N, N))
    nnodes = length(grid.nodes)
    free = findall(node -> norm(node.x, Inf) < 1-0.01/N, grid.nodes)
    A, b = assemble_dirichlet_p1(grid, f_ex)
    U = zeros(nnodes)
    U[free] = A[free, free] \ b[free];    
    return U, grid
end

# the next function implements the two error norms
function errors(U, grid)
    nnodes = length(grid.nodes)
    err_L2_sq = 0.0 
    err_H1_sq = 0.0 
    
    for T in grid.cells 
        # local information (see comments in code above)
        XT = (grid.nodes[T.nodes[1]].x, grid.nodes[T.nodes[2]].x, grid.nodes[T.nodes[3]].x)
        F = [ XT[2] - XT[1] XT[3] - XT[1] ]
        volT = abs(det(F))
        F⁻ᵀ = pinv(F)'  
        ξ = sum(XT) / 3 
        ∇ψ̂ = ( [-1.0, -1.0], [1.0, 0.0], [0.0, 1.0] )
        ∇ψ = ( F⁻ᵀ * ∇ψ̂[1], F⁻ᵀ * ∇ψ̂[2], F⁻ᵀ * ∇ψ̂[3] )
        ψ = (1/3, 1/3, 1/3)

        # now we evaluate u, ∇u at the quadrature point ξ
        uξ = sum( U[nᵢ] * ψ[i] for (i, nᵢ) in enumerate(T.nodes) )
        ∇uξ = sum( U[nᵢ] * ∇ψ[i] for (i, nᵢ) in enumerate(T.nodes) )
        
        # from those we can now update the errors
        err_L2_sq += volT * (u_ex(ξ) - uξ)^2
        err_H1_sq += volT * norm(∇u_ex(ξ) - ∇uξ)^2
    end
    
    return sqrt(err_L2_sq), sqrt(err_H1_sq)
end

In [ ]:
# now we can compute the errors in a nice simple loop
NN = [5, 10, 20, 40, 80, 160]
errs0 = Float64[]
errs1 = Float64[] 
for N in NN 
    err0, err1 = errors(solve(N)...)
    push!(errs0, err0); push!(errs1, err1)
end


We are now in a position to produce a nice figure. The following is still quite simple but close to publication quality. Some key features: 
- label all axis
- appropriate fonts and font sizes
- clearly label the graphs
- use a modern colour scheme
- indicate the convergence rates

In [ ]:
# ... and visualize them
fig = Figure(size = (400, 350); fontsize=14)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"\text{error}", 
          xscale = log10, yscale = log10,)
scatterlines!(NN, errs0; linewidth=2, markersize=12, label=L"L^2")
scatterlines!(NN, errs1; linewidth=2, markersize=12, label = L"H^1")
NN1 = NN[3:5]
lines!(NN1, 2.2 ./ NN1; color=:black, linewidth=2, linestyle = :dash, label = L"\sim h, h^2")
lines!(NN1, 3 ./ NN1.^2; color=:black, linewidth=2, linestyle = :dash)
axislegend(ax)
fig

The last results give us a lot of confidence that (1) our method converges. And (2) it tells us what convergence rates to expect: $O(h)$ in energy-norm and $O(h^2)$ in $L^2$-norm. We will rigorously prove these. 

**WARNING:** The method of manufactured solutions must be used with care! If we assume more in our postulated solution than is typically satisfied then we might get spurious results - either better or worse than they should be. We will return to this!